## EuropePMC API for remainder

This will get as many of the reamaining unmapped PMIDs as possible from the EurpoPMC API.

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
import requests
import json

In [2]:
still_no_map = pickle.load(open('../data/no_map_NLM.pkl', 'rb'))
new_map = {}

In [3]:
URL="https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=ext_id:{}%20src:med&format=json"

In [4]:
def get_date(result):
    res = result['resultList']['result']
    if len(res) == 1 and type(res[0]) == dict:
        return res[0].get('firstPublicationDate', None)
    elif len(res) > 1 and type(res[0]) == dict:
        dates = []
        for r in res:
            date = res[0].get('firstPublicationDate', '9999-99-99')
            dates.append(date)
        return min(dates)
    
    return None

Saving this for last because I don't want to hammer the API... but... looking at the docs:

https://europepmc.org/RestfulWebService

I don't see a way to do a batch query...

In [5]:
# Slow, but only way to get is to query API individually...
for pmid in tqdm(still_no_map):
    r = requests.get(URL.format(pmid))
    result = json.loads(r.text)
    date = get_date(result)
    new_map[pmid] = date

 12%|█▏        | 49793/425698 [8:27:19<67:12:25,  1.55it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 15%|█▍        | 63806/425698 [10:50:14<63:09:30,  1.59it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 18%|█▊        | 77741/425698 [13:14:20<58:59:33,  1.64it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
len(new_map)

360736

In [21]:
# Slow, but only way to get is to query API individually... For some reason it stopped. So restarted it.
for pmid in tqdm(list(still_no_map)[len(new_map)+1:]):
    r = requests.get(URL.format(pmid))
    result = json.loads(r.text)
    date = get_date(result)
    new_map[pmid] = date

100%|██████████| 64961/64961 [11:00:54<00:00,  1.64it/s]  


In [23]:
new_map = {k: v for k, v in new_map.items() if v is not None}
print('Mapped: {:,}'.format(len(new_map)))

final_no_map = still_no_map - set(new_map.keys())
print('Still unmapped: {:,}'.format(len(final_no_map)))

Mapped: 406,631
Still unmapped: 19,067


In [24]:
pickle.dump(new_map, open('../data/pmid_to_year_EBI.pkl', 'wb'))
pickle.dump(final_no_map, open('../data/no_map_EBI.pkl', 'wb'))